In [1]:
import tkinter as tk
import pandas as pd
from tkinter import font as tkFont
from tkinter import ttk
from PIL import Image, ImageTk
from river import (
    stream,
    compose,
    preprocessing,
    evaluate,
    metrics,
    linear_model,
    imblearn,
    stats,
    optim 
)
import numbers
import numpy as np

### 데이터 불러오기

In [2]:
#도시별 데이터
reg = pd.read_csv('데이터셋_v12.csv', encoding = 'cp949')
data = reg.iloc[:,2:]
data["도시"] = 0
data["동반"] = 0

compan = reg.iloc[:,4:9]

#유사도
sim = pd.read_csv("JSscore_해외국내도시.csv", encoding='cp949')
domestic =  pd.DataFrame()
domestic['city_province'] = sim['광역'] + '-' + sim['기초']

#공간가중치
spatial = pd.read_csv("공간가중행렬.csv", encoding='cp949')

### 추천 시스템 

In [4]:
window = tk.Tk()
window.title("Recommendation")
window.configure(bg='lightblue')

# 폰트 설정
helv36 = tkFont.Font(family='Helvetica', size=14, weight=tkFont.BOLD)
helv26 = tkFont.Font(family='Helvetica', size=12)

# 입력 칸 생성
city_label = tk.Label(window, text="Enter the city:", bg='lightblue', fg='white', font=helv36)
city_label.grid(row=0, column=0, padx=5, pady=5)

city_entry = tk.Entry(window)
city_entry.grid(row=0, column=1, padx=5, pady=5)

companion_label = tk.Label(window, text="Enter the companion type:", bg='lightblue', fg='white', font=helv36)
companion_label.grid(row=1, column=0, padx=5, pady=5)

companion_entry = tk.Entry(window)
companion_entry.grid(row=1, column=1, padx=5, pady=5)

# 선호도 입력
preferences = []
good=['불만족','만족']

column = ['거리','언급 트렌드','6월 계절트렌드','여행지 지명도','해외 유사도','동반 유형']

for i in range(6):
    frame = tk.Frame(window)
    frame.grid(row=i+2, column=0, padx=5, pady=5)

    label = tk.Label(frame, text=f"{column[i]} 선호도:")
    label.pack(side="left")

    var = tk.IntVar()
    preferences.append(var)

    for val in range(-2, 3):
        rb = tk.Radiobutton(frame, text=str(val), variable=var, value=val)
        rb.pack(side="left")

model = linear_model.LogisticRegression(optim.SGD(lr=0.01))
metric = metrics.ROCAUC()

input_data = []

# 추천 시스템
def calculate_data():
    global data
    
    output_label.config(text="")
    modified_output_label.config(text="")

    # 희망도시 입력값 불러오기
    city = city_entry.get()
    if city in sim.columns:
        data.iloc[:,-2] = sim[city]

    # 동반자 유형 입력값 불러오기
    companion = companion_entry.get()
    if companion in compan.columns:
        data.iloc[:,-1] = compan[companion]

    #데이터, 공간가중행렬 inner product  
    data = pd.DataFrame(np.dot(spatial, data))
    
    # 선호도 입력값 불러오기
    prefs = []
    for var in preferences:
        pref = var.get()
        prefs.append(pref)

    # 선호도와 각 요소별 데이터 가중합 구하기
    result = [0,0,0,0,0,0]  
    for i in range(6): 
        result[i] = data.iloc[:,i] * prefs[i] 

    result = pd.DataFrame(result)
    col_sums = [result[col].sum() for col in result.columns]

    domestic['선호도결과'] = col_sums
    global sorted_Re
    sorted_Re = domestic.sort_values('선호도결과', ascending=False)
    
    global modified2
    modified2 = sorted_Re
    output_text = "Top 10 rows:\n"
       
    #추천 만족 입력받기
    for i in range(10):
        output_text = str(sorted_Re.iloc[i,0])
        out = tk.Frame(window)
        out.grid(row=i+11, column=0, padx=5, pady=5)
        
        outlabel = tk.Label(out, text= str(i+1) + '.' + output_text + " | 추천 만족도를 입력해주세요:")
        outlabel.pack(side="left")
        
        love = tk.IntVar()
        input_data.append(love)
        
        for val in range(2):
            iloveit = tk.Radiobutton(out, text=good[val], variable=love, value=val)
            iloveit.pack(side="left")

            
calculate_button = tk.Button(window, text="1차 입력", command=calculate_data, bg='blue', fg='white', font=helv26)
calculate_button.grid(row=8, column=0, columnspan=2, padx=5, pady=5)

output_frame = tk.Frame(window, bg='lightblue')
output_frame.grid(row=9, column=0, columnspan=2, padx=5, pady=5)

output_label = tk.Label(output_frame, text="", bg='lightblue', fg='white', font=helv36) 
output_label.pack()

# 추천 만족도 반영 함수
def modified_calculate_data():
    global modified2
    global input_data
    global model, metric

    loves = []
    for var in input_data:
        pref = var.get()
        loves.append(pref)
        
    y_true = pd.Series(loves)
    ind = modified2.iloc[0:10,:].index
    X = data.loc[ind]  #상위 10개의 결과값이 담긴 dataframe 생성
    
    for xi, yi in stream.iter_pandas(X, y_true):
        yi_pred = model.predict_one(xi)
        metric.update(yi,yi_pred)
        model.learn_one(xi, yi)

    modified = model.predict_proba_many(data)
    modified_data = pd.concat([domestic, modified], axis = 1).sort_values(True, ascending=False)
    
    modified2 = modified_data
    
    input_data = []
    
    for widget in output_frame.pack_slaves():
        widget.pack_forget()
    
    for widget in modified_output_frame.pack_slaves():
        widget.pack_forget()

    # 새로운 도시 추천(만족도 반영)
    for i in range(10):
        city_name = modified_data.iloc[i,0]
        frame = tk.Frame(modified_output_frame)
        frame.pack()

        label = tk.Label(frame, text= str(i+1) + '.' + city_name + " | 추천 만족도를 입력해주세요:")
        label.pack(side="left")

        love = tk.IntVar()
        input_data.append(love)

        for val in range(2):
            iloveit = tk.Radiobutton(frame, text=good[val], variable=love, value=val)
            iloveit.pack(side="left")
            
modified_output_frame = tk.Frame(window)
modified_output_frame.grid(row=9, column=0, columnspan=2, padx=5, pady=5)

modified_output_label = tk.Label(modified_output_frame, text="")
modified_output_label.pack()

calculate_button2 = tk.Button(window, text="Calculate", command=modified_calculate_data)
calculate_button2.grid(row=8, column=1, columnspan=2, padx=5, pady=5)

window.mainloop()